In [42]:
import pandas as pd
import os, sys
import numpy as np

In [77]:
with open(os.path.join(sys.path[0], "AoC_Day4_passports.txt"), "r") as f: ##open file
    contents = f.read()
    passgroup = contents.split("\n\n")

In [78]:
passlst= []
for pp in passgroup:
    pp_flds = pp.split()
    passlst.append(pp_flds)

In [79]:
#     byr (Birth Year)
#     iyr (Issue Year)
#     eyr (Expiration Year)
#     hgt (Height)
#     hcl (Hair Color)
#     ecl (Eye Color)
#     pid (Passport ID)
#     cid (Country ID)

flds = ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid', 'cid']
df = pd.DataFrame(columns = flds)

for pp in passlst:
    record = []
    for fld in flds:
        match = [s.split(':')[1] for s in pp if fld in s]
        if match:
            record.append(match[0])
        else:
            record.append(np.nan)
      
    x = pd.Series(record, index = flds) 
    df = df.append(x, ignore_index=True)

In [80]:
df['Nullct'] = df.isnull().sum(axis = 1)
df['cidNullCheck'] = df['cid'].isnull()

In [81]:
df['valid1'] = ((df['cidNullCheck'] == False) & (df['Nullct'] == 0)) | ((df['cidNullCheck'] == True) & (df['Nullct'] == 1))

In [82]:
print( df['valid1'].value_counts()[True] )

190


You can continue to ignore the cid field, but each other field has strict rules about what values are valid for automatic validation:

   - byr (Birth Year) - four digits; at least 1920 and at most 2002. (complete)
   - iyr (Issue Year) - four digits; at least 2010 and at most 2020.(complete)
   - eyr (Expiration Year) - four digits; at least 2020 and at most 2030.(complete)
   - hgt (Height) - a number followed by either cm or in:   (complete)<br>  
        - If cm, the number must be at least 150 and at most 193.<br>
        - If in, the number must be at least 59 and at most 76.
   - hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
   - ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth. (complete)
   - pid (Passport ID) - a nine-digit number, including leading zeroes. (complete)
   - cid (Country ID) - ignored, missing or not.


In [87]:
df['v_byr'] = (1920 <= pd.to_numeric(df['byr'], errors='coerce')) & (pd.to_numeric(df['byr'], errors='coerce') <= 2002)
df['v_iyr'] = (2010 <= pd.to_numeric(df['iyr'], errors='coerce')) & (pd.to_numeric(df['iyr'], errors='coerce') <= 2020)
df['v_eyr'] = (2020 <= pd.to_numeric(df['eyr'], errors='coerce')) & (pd.to_numeric(df['eyr'], errors='coerce') <= 2030)
df['hgt_num'] =  df['hgt'].str.extract('(^\d*)')
df['hgt_units'] = df['hgt'].replace('(\d)', '', regex=True)
df['v_hgt'] = ( \
                (pd.notna(df['hgt']) & (df['hgt_units'] == 'in') & \
                      ((59<= pd.to_numeric(df['hgt_num'], errors='coerce')) & \
                       (pd.to_numeric(df['hgt_num'], errors='coerce')<=76))) \
                | \
                (pd.notna(df['hgt']) & (df['hgt_units'] == 'cm') & \
                      ((150<= pd.to_numeric(df['hgt_num'], errors='coerce')) & \
                       (pd.to_numeric(df['hgt_num'], errors='coerce')<=193))) \
               )

df['v_hcl']= (df['hcl'].str.contains(r'^#(?:[0-9a-fA-F]{3}){1,2}$'))
df['v_ecl'] = df['ecl'].isin(['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'])
df['v_pid'] = (df['pid'].str.len() == 9) & df['pid'].str.isnumeric()

In [93]:
#final check for validity in condition 2 for day 4
df['valid2'] = df['v_byr'] & df['v_iyr'] & df['v_eyr'] & df['v_hgt']& df['v_hcl'] & df['v_ecl'] & df['v_pid']

In [94]:
print( df['valid2'].value_counts()[True] )

121
